In [18]:
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import pandas as pd


filename = "Waves_2023.txt"
file = pd.DataFrame(pd.read_csv(filename, delim_whitespace=True))

df = file[['WVHT', 'APD', 'DPD', 'MWD', 'WTMP']].replace([9999.0,999.0, 99.0, 'NA'], np.nan).dropna()
df.shape
X = np.column_stack((df['APD'],df['DPD'],df['MWD'],df['WTMP']))
y = df['WVHT'].to_numpy()

/var/folders/bm/1zbxfdz545vb2vms38bdffv80000gn/T/ipykernel_84705/2497530378.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file = pd.DataFrame(pd.read_csv(filename, delim_whitespace=True))


In [19]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

mlp_model = MLPRegressor(hidden_layer_sizes=(16, 16, 16),activation='logistic')
mlp_model.fit(X_train, y_train)
mlp_y_pred = mlp_model.predict(X_test)
mlp_mse = mean_squared_error(y_test, mlp_y_pred)
print('MLP MSE:', mlp_mse)

baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)
baseline_y_pred = baseline_model.predict(X_test)
baseline_mse = mean_squared_error(y_test, baseline_y_pred)
print('Baseline MSE:', baseline_mse)


MLP MSE: 0.16492317154126626
Baseline MSE: 0.18773177187893833


In [20]:
# 1. The Baseline model is better than the MLP model
# 2. The cost fucntion of the MLP model is the mean squared error

In [21]:
layers = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5]
nodes = [1, 16, [1, 16], [16, 16], [16, 4, 16], [16, 16, 16], [16, 16, 16, 16], [16, 32, 64, 128], [1, 1, 1, 1, 1], [100, 100, 100, 100, 10]]
train_errors = []
test_errors = []

for layer, node in zip(layers, nodes):
    mlp_model = MLPRegressor(hidden_layer_sizes=node)
    mlp_model.fit(X_train, y_train)
    mlp_train_pred = mlp_model.predict(X_train)
    mlp_test_pred = mlp_model.predict(X_test)
    train_error = mean_squared_error(y_train, mlp_train_pred)
    test_error = mean_squared_error(y_test, mlp_test_pred)
    train_errors.append(train_error)
    test_errors.append(test_error)

print("Layers\tNodes\tTrain Error\tTest Error")
for layer, node, train_error, test_error in zip(layers, nodes, train_errors, test_errors):
    print(f"{layer}\t{node}\t{train_error:.4f}\t\t{test_error:.4f}")



Layers	Nodes	Train Error	Test Error
1	1	0.2990		0.2817
1	16	0.2058		0.1968
2	[1, 16]	0.2817		0.2601
2	[16, 16]	0.2000		0.1920
3	[16, 4, 16]	0.2439		0.2274
3	[16, 16, 16]	0.2180		0.2109
4	[16, 16, 16, 16]	0.1787		0.1756
4	[16, 32, 64, 128]	0.1865		0.1797
5	[1, 1, 1, 1, 1]	0.2817		0.2600
5	[100, 100, 100, 100, 10]	0.1789		0.1760


In [22]:
# 4. The best model is the one with 4 layers and 16 nodes in each layer. Having layers with only 1 node in each layers make the worst models. The best model is more layers and more nodes in each layer.

# 5. After using a 'logistic' activation function, the best model is the model with 4 layers and varying # of nodes. The baseline is still better than the best model.